In [54]:
import math
import torch
import gpytorch
from torch import optim
from torch.autograd import Variable
from gpytorch.kernels import RBFKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.random_variables import GaussianRandomVariable
from matplotlib import pyplot as plt

In [55]:
# Simple training data: let's try to learn a sine function, but with KISS-GP let's use 100 training examples.
def make_data(cuda=False):
    train_x = Variable(torch.linspace(0, 1, 50000))
    train_y = Variable(torch.sin(train_x.data * (2 * math.pi)))
    test_x = Variable(torch.linspace(0, 1, 510))
    test_y = Variable(torch.sin(test_x.data * (2 * math.pi)))
    if cuda:
        train_x = train_x.cuda()
        train_y = train_y.cuda()
        test_x = test_x.cuda()
        test_y = test_y.cuda()
    return train_x, train_y, test_x, test_y

In [56]:
# All tests that pass with the exact kernel should pass with the interpolated kernel.
class LatentFunction(gpytorch.GridInducingPointModule):
    def __init__(self):
        super(LatentFunction, self).__init__(grid_size=100, grid_bounds=[(0,1000), (0, 2000)])
        self.mean_module = ConstantMean(constant_bounds=[-1e-5, 1e-5])
        self.covar_module = RBFKernel(log_lengthscale_bounds=(-5, 6))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        latent_pred = GaussianRandomVariable(mean_x, covar_x)
        return latent_pred


class GPRegressionModel(gpytorch.GPModel):
    def __init__(self):
        super(GPRegressionModel, self).__init__(GaussianLikelihood())
        self.latent_function = LatentFunction()

    def forward(self, x):
        return self.latent_function(x)

In [ ]:
def test_kissgp_gp_mean_abs_error(train_x, train_y, test_x, test_y):
    gp_model = GPRegressionModel()

    # Optimize the model
    gp_model.train()
    optimizer = optim.Adam(gp_model.parameters(), lr=0.1)
    optimizer.n_iter = 0
    for i in range(25):
        optimizer.zero_grad()
        output = gp_model(train_x)
        loss = -gp_model.marginal_log_likelihood(output, train_y)
        loss.backward()
        optimizer.n_iter += 1
        optimizer.step()

    # Test the model
    gp_model.eval()
    gp_model.condition(train_x, train_y)
    test_preds = gp_model(test_x).mean()
    mean_abs_error = torch.mean(torch.abs(test_y - test_preds))
    print(mean_abs_error.data.squeeze()[0])

#     assert(mean_abs_error.data.squeeze()[0] < 0.05)



In [ ]:
import numpy as np
import csv

training_b = np.loadtxt("ubbase.txt", dtype = int, usecols=(0, 1, 2))
test_b = np.loadtxt("ubtest.txt", dtype = int, usecols=(0, 1, 2))

train_x = torch.from_numpy(training_b[:
                                      ,0:2])
train_x = train_x.float()
train_x = Variable(train_x)

train_y = torch.from_numpy(training_b[:,-1])
train_y = train_y.float()
train_y = Variable(train_y)

test_x = torch.from_numpy(test_b[:,0:2])
test_x = test_x.float()
test_x = Variable(test_x)

test_y = torch.from_numpy(test_b[:,-1])
test_y = test_y.float()
test_y = Variable(test_y)

test_kissgp_gp_mean_abs_error(train_x, train_y, test_x, test_y)